# saving out mphi masks

In [8]:
from csbdeep.utils.tf import limit_gpu_memory
limit_gpu_memory(fraction = 0.00, allow_growth=False, total_memory=49140)

In [1]:
import napari
from skimage import io
import btrack
import os
from tqdm.auto import tqdm
import numpy as np
import btrack
from glob import glob
from homuncu_loc.dataio import find_files_with_basename, ID_extractor

#### Define image fn 

In [2]:
# print gpu information
!nvcc --version
!nvidia-smi

# load cellpose
# from cellpose import core, utils, models, metrics
import matplotlib.pyplot as plt 
from skimage.morphology import remove_small_objects

# check to see if GPU can be used
# use_GPU = core.use_gpu()
# yn = ['NO', 'YES']
# print(f'>>> GPU activated? {yn[use_GPU]}')

# # define segmentation model parameters
# model = models.Cellpose(gpu=use_GPU, 
#                         model_type='cyto') # cytoplasmic segmentation 
# channels = [0,0] # this means using a grayscale image for both nuclei and cyto channels (even if not using nuclei, still have to say its same colour [greyscale = 0])

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Mon Oct  9 09:42:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:65:00.0  On |                  Off |
| 30%   44C    P5    38W / 300W |  45557MiB / 49140MiB |     29%      Default |
|                               |                      |                  N/A |
+

# define a general segmentation framework for macroph and edit params for other cell

In [16]:
from tqdm.auto import tqdm
from cellpose import models
from skimage.morphology import remove_small_objects
import numpy as np
import btrack

def segment_channel(segmentation_channel, diameter=60, flow_threshold=0.99, channels=[0,0], cellprob_threshold=-2, min_mask_size=500, return_diams=False, start_point=None, end_point=None):
    """
    Segments a specified channel of an image using Cellpose and optionally returns the average diameter of segments.

    Args:
    - segmentation_channel (ndarray): 3D array of the target channel where the 3rd dimension represents the Z-stack.
    - diameter (int, optional): Average cell diameter for Cellpose. Default is 60.
    - flow_threshold (float, optional): Threshold for flow in Cellpose. Default is 0.99.
    - channels (list, optional): Channel input for Cellpose. Default is [0,0].
    - cellprob_threshold (float, optional): Cell probability threshold for Cellpose. Default is -2.
    - min_mask_size (int, optional): Minimum size of objects to keep in the segmented mask. Default is 500.
    - return_diams (bool, optional): If set to True, the function will also return the average diameter of segments.
    - start_point (int, optional): Starting point for segmentation frames. Default is None.
    - end_point (int, optional): Ending point for segmentation frames. Default is None.

    Returns:
    - ndarray: 3D mask array with the same shape as the input, where segmented regions are labeled.
    - float (optional): Average diameter of segments if return_diams is True.
    """

    if start_point is None or end_point is None:
        # Calculate intensities for each frame in Z-stack
        intensities = [np.sum(frame) for frame in tqdm(segmentation_channel, desc='finding where segmentation channel is expressed in Z')]

        # Estimate the background level
        background_level = np.min(intensities)
        threshold = background_level + ((np.max(intensities)-np.min(intensities))*0.50)

        if start_point is None:
            # Find the points where the curve starts to deviate from the background
            start_point = np.argmax(intensities > threshold)

        if end_point is None:
            end_point = len(intensities) - np.argmax(intensities[::-1] > threshold) - 1

    # Initialize output mask array
    mask_array = np.zeros_like(segmentation_channel)

    # Initialize the model
    model = models.Cellpose(model_type='cyto')
    
    # List to store diameters if return_diams is True
    diams = []

    # Segment over selected frames
    for n, frame in tqdm(enumerate(segmentation_channel[start_point:end_point]), total=end_point-start_point, desc='segmenting over select frames'):
        n += start_point
        masks, _, _, diam = model.eval(frame, diameter=diameter, flow_threshold=flow_threshold, channels=channels, cellprob_threshold=cellprob_threshold)
        masks = remove_small_objects(masks, min_size=min_mask_size)
        mask_array[n] = masks
        diams.append(diam)

    if return_diams:
        return mask_array, diams
    else:
        return mask_array

In [3]:
from stardist.models import StarDist2D
from csbdeep.utils import normalize
import numpy as np
from tqdm.auto import tqdm
from skimage import segmentation
from skimage.morphology import remove_small_objects
import btrack
axis_norm = (0,1) 

def segmestimate_channel(segmentation_channel, min_mask_size=100, start_point=None, end_point=None):
    """
    Segments a specified channel of an image using stardist and estimates an approximation of cytoplasmic area by expanding the nucleus.

    Args:
    - segmentation_channel (ndarray): 3D array of the target channel where the 3rd dimension represents the Z-stack.
   
    - start_point (int, optional): Starting point for segmentation frames. Default is None.
    - end_point (int, optional): Ending point for segmentation frames. Default is None.

    Returns:
    - ndarray: 3D mask array with the same shape as the input, where segmented regions are labeled.
    - float (optional): Average diameter of segments if return_diams is True.
    """

    if start_point is None or end_point is None:
        # Calculate intensities for each frame in Z-stack
        intensities = [np.sum(frame) for frame in tqdm(segmentation_channel, desc='finding where segmentation channel is expressed in Z')]

        # Estimate the background level
        background_level = np.min(intensities)
        threshold = background_level + ((np.max(intensities)-np.min(intensities))*0.50)

        if start_point is None:
            # Find the points where the curve starts to deviate from the background
            start_point = np.argmax(intensities > threshold)

        if end_point is None:
            end_point = len(intensities) - np.argmax(intensities[::-1] > threshold) - 1

    # Initialize output mask array
    mask_array = np.zeros_like(segmentation_channel)

    # Initialize the model
    model = StarDist2D.from_pretrained('2D_versatile_fluo')

    # Segment over selected frames
    for n, frame in tqdm(enumerate(segmentation_channel[start_point:end_point]), total=end_point-start_point, desc='segmenting over select frames'):
        n += start_point
        frame = normalize(frame, 1, 99.8, axis=axis_norm)
        masks, details = model.predict_instances(frame)
        masks = remove_small_objects(masks, min_size=min_mask_size)
        masks = segmentation.expand_labels(masks, distance=20)
        mask_array[n] = masks
   
    return mask_array

2023-10-09 09:42:43.928119: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-09 09:42:43.990647: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-09 09:42:43.990685: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-09 09:42:43.990714: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-09 09:42:44.001408: I tensorflow/core/platform/cpu_feature_g

In [5]:
!nvidia-smi

Mon Oct  9 09:44:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:65:00.0  On |                  Off |
| 30%   45C    P5    38W / 300W |    779MiB / 49140MiB |     29%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
props = ('axis_major_length', # first define some cell properties 
         'axis_minor_length', # this helps improve the tracking
         'eccentricity',      # by comparing similarities between slices/frames
         'area', 
         'orientation',
         'mean_intensity', 
        )

In [7]:
basedir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/images'

In [8]:
image_basenames = [
    # "20230705_40X_23-01-001B3_Multichannel Z-Stack_20230705_1298",
    # "20230705_40X_23-01-001B3_Multichannel Z-Stack_20230705_1299",
    # "20230705_40X_23-01-001B3_Multichannel Z-Stack_20230705_1300",
    # "20230705_40X_23-01-005A3_Multichannel Z-Stack_20230705_1306",
    # "20230705_40X_23-01-005A3_Multichannel Z-Stack_20230705_1307",
    # "20230705_40X_23-01-005A3_Multichannel Z-Stack_20230705_1308",
    # "20230705_40X_23-01-005A3_Multichannel Z-Stack_20230705_1309",
    # "20230705_40X_23-01-005A3_Multichannel Z-Stack_20230705_1310",
    # "20230705_40X_23-01-005A3_Multichannel Z-Stack_20230705_1311",
    # "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313",
    # "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1314",
    # "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1315",
    # "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1316",
    # "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1317",
    # "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1318",
    # "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1319",
    # "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1320",
    # "20230707_40X_23-01-005A3_Multichannel Z-Stack_20230707_1324",
    # "20230707_40X_23-01-005A3_Multichannel Z-Stack_20230707_1325",
    # "20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1340",
    # "20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1341",
    # "20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1342",
    # "20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1343",
    # "20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1344",
    # "20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1361",
    # "20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1362",
    # "20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1363",
    # "20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1364",
    # "20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1365",
    #    "20230801_20X_23-03-001B4_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230801_1463",
    # "20230801_20X_23-03-001B4_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230801_1464",
    # "20230801_20X_23-03-001B4_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230801_1465",
    # "20230801_20X_23-03-001B4_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230801_1467",
    # "20230801_20X_23-03-001B2_DAPI_ZO-1_Ki67_Mtb_Multichannel Z-Stack_20230801_1482",
    # "20230801_20X_23-03-001B2_DAPI_ZO-1_Ki67_Mtb_Multichannel Z-Stack_20230801_1483",
    # "20230801_20X_23-03-001B2_DAPI_ZO-1_Ki67_Mtb_Multichannel Z-Stack_20230801_1484",
    # "20230802_20X_23-03-001A4_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1500",
    # "20230802_20X_23-03-001A4_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1501",
    # "20230802_20X_23-03-001A1_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1510",
    # "20230802_20X_23-03-001A1_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1511",
    # "20230802_20X_23-03-001A1_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1512",
    # "20230802_20X_23-03-001A1_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1513",
    # "20230802_20X_23-03-001A1_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1514",
    # "20230801_20X_23-03-001B2_DAPI_ZO-1_Ki67_Mtb_Multichannel Z-Stack_20230801_1481"
    '20230802_20X_23-03-001A2_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230802_1504', 
    '20230802_20X_23-03-001A5_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230802_1496',
    '20230801_20X_23-03-001B3_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230801_1468',
    '20230801_20X_23-03-001B6_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230801_1453',
]
    
    

In [9]:
image_fns = [os.path.join(basedir, fn+'.tif') for fn in image_basenames]

In [11]:
image_fns = [f'{fn}.tif' for fn in [
    "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/images/20230801_20X_23-03-001B6_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230801_1453",
    "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/images/20230801_20X_23-03-001B3_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230801_1468",
    "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_Ki67_MTB/images/20230801_20X_23-03-001B2_DAPI_ZO-1_Ki67_Mtb_Multichannel Z-Stack_20230801_1481",
    "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_Ki67_MTB/images/20230801_20X_23-03-001B2_DAPI_ZO-1_Ki67_Mtb_Multichannel Z-Stack_20230801_1482",
    "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_Ki67_MTB/images/20230801_20X_23-03-001B2_DAPI_ZO-1_Ki67_Mtb_Multichannel Z-Stack_20230801_1483",
    "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_Ki67_MTB/images/20230801_20X_23-03-001B2_DAPI_ZO-1_Ki67_Mtb_Multichannel Z-Stack_20230801_1484",
    "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/images/20230802_20X_23-03-001A5_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230802_1496",
    "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/images/20230802_20X_23-03-001A2_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230802_1504"
]]


In [12]:
from macrohet.notify import send_sms

In [13]:
# Iterate over a list of image file paths using tqdm for progress tracking
for image_fn in tqdm(image_fns, total=len(image_fns), desc='iterating over image volumes'):

    # if not ID_extractor(image_fn) in ['1463']:#, '1317', '1319']:
    #     print(f'{image_fn} skipped')
    #     continue
    # Define the output filename
    output_fn = image_fn.replace('images', 'sc_analyses').replace('.tif', '_mphi.h5').replace('/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/', 
                                                                                         '/home/dayn/data/homuncu_loc_temp/seperate_tracks/')
    # if os.path.exists(output_fn):
    #     continue
    print(image_fn)
    # if image_fn != '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/images/20230801_20X_23-03-001B6_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230801_1453.tif':
    try:
        # Read the image using imageio.imread
        image = io.imread(image_fn)
    except Exception as e:
        # Code to handle the exception and print the error message
        print(e)
        continue
    # # Extract the third channel (mphi_channel) from the image
    # mphi_channel = image[..., 2]
    
    # # Segment the mphi_channel using custom function 'segment_channel'
    # mphi_masks, mphi_diams = segment_channel(
    #     mphi_channel,
    #     diameter=60,
    #     flow_threshold=0.99,
    #     channels=[0, 0],
    #     cellprob_threshold=-2,
    #     min_mask_size=200,
    #     return_diams=True,
    #     start_point=None,
    #     end_point=None
    # )
    # masks = mphi_masks

    # Extract the 0th channel (nuclear) from the image
    nuc_channel = image[..., 0]
    
    # Segment the nuc_channel using custom function 'segmestimate_channel' to get the perinuclear
    perinuc_masks = segmestimate_channel(
        nuc_channel,
        min_mask_size=100,
        start_point=None,
        end_point=None
    )
    masks = perinuc_masks

    # # update
    # send_sms(f'mphi seg done {ID_extractor(image_fn)}')
    
    # # Extract the second channel (zo1_channel) from the image
    # zo1_channel = image[..., 1]
    
    # # Segment the zo1_channel for iat2 using 'segment_channel' with specific parameters
    # iat2_masks, iat2_diams = segment_channel(
    #     zo1_channel,
    #     diameter=None,
    #     flow_threshold=0,
    #     channels=[0, 0],
    #     cellprob_threshold=0,
    #     min_mask_size=200,
    #     return_diams=True,
    #     start_point=None,
    #     end_point=None
    # )
    
    # # update
    # send_sms(f'iat2 seg done {ID_extractor(image_fn)}')
    
    # # Segment the zo1_channel for iat1 using 'segment_channel' with specific parameters
    # iat1_masks, iat1_diams = segment_channel(
    #     zo1_channel,
    #     diameter=150,
    #     flow_threshold=0.99,
    #     channels=[0, 0],
    #     cellprob_threshold=-3,
    #     min_mask_size=2500,
    #     return_diams=True,
    #     start_point=None,
    #     end_point=None
    # )
    
    # # update
    # send_sms(f'iat1 seg done {ID_extractor(image_fn)}')
    
    # # Loop over three different mask types: mphi_masks, iat2_masks, iat1_masks
    # for n, masks in tqdm(enumerate([mphi_masks, iat2_masks, iat1_masks]), total=3):
        
    # Convert the segmentation masks to objects using btrack
    objects = btrack.utils.segmentation_to_objects(
        segmentation=perinuc_masks,
        intensity_image=image,
        properties=props,
        use_weighted_centroid=False,
        assign_class_ID=True,
    )

    # update
    # send_sms(f'object localisation {n, ID_extractor(image_fn)} done')

    
    # Check if mtb infected above threshold and measure mtb properties for each cell
    threshold = 230
    mtb_ch = 3
    
    for o in tqdm(objects, desc='Measuring Mtb properties of each cell'):
        coordinates = np.argwhere(masks[o.t] == o.properties['class_id'])
        pixel_values = image[o.t, coordinates[:, 0], coordinates[:, 1]]
        mtb_status = np.any(pixel_values[:,mtb_ch] > threshold)
        mtb_area = np.sum(pixel_values[:,mtb_ch] > threshold)
        o.properties['mtb_status'] = mtb_status
        o.properties['mtb_area'] = mtb_area
        # o.properties['pixel_values'] = pixel_values

    # update
    # send_sms(f'mtb props {n, ID_extractor(image_fn)} done')

    # Initialize BayesianTracker
    with btrack.BayesianTracker() as tracker:
        
        # Configure the tracker using a config file
        tracker.configure('/home/dayn/analysis/models/loc.json')
        
        # Set max search radius to a very limited radius
        tracker.max_search_radius = 5
        
        # Define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        
        # Use visual features to track
        tracker.features = props
        
        # Append the objects to be tracked
        tracker.append(objects)
        
        # Set the volume
        tracker.volume = ((0, masks.shape[1]), (0, masks.shape[2]), (-1e5, 1e5))
        
        # Track them (in interactive mode)
        tracker.track(step_size=10)
        
        # Generate hypotheses and run the global optimizer
        tracker.optimize()
        
        # Get the tracks as a Python list
        tracks = tracker.tracks

    # update
    # send_sms(f'tracking {n, ID_extractor(image_fn)} done')

    # Define the output filename
    output_fn = image_fn.replace('images', 'sc_analyses').replace('.tif', '_mphi.h5').replace('/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/', 
                                                                                         '/home/dayn/data/homuncu_loc_temp/seperate_tracks/')
    os.makedirs(os.path.dirname(output_fn), exist_ok=True)
    
    # Define the object type based on the iteration index
    obj_type = f'obj_type_{1}'


    # Write the tracks and segmentation masks to an HDF5 file using btrack.io.HDF5FileHandler
    with btrack.io.HDF5FileHandler(output_fn, 'w', obj_type=obj_type) as writer:
        writer.write_tracks(tracks)
        writer.write_segmentation(masks)

    # update
    # send_sms(f'tracks saved {n, ID_extractor(image_fn)}')


iterating over image volumes:   0%|          | 0/8 [00:00<?, ?it/s]

/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/images/20230801_20X_23-03-001B6_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230801_1453.tif


finding where segmentation channel is expressed in Z:   0%|          | 0/233 [00:00<?, ?it/s]

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


segmenting over select frames:   0%|          | 0/176 [00:00<?, ?it/s]

...

[INFO][2023/10/09 12:17:10 PM] Localizing objects from segmentation...
progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 81/81 [00:23<00:00,  3.50it/s]
[INFO][2023/10/09 12:17:33 PM] Objects are of type: <class 'dict'>
[INFO][2023/10/09 12:17:34 PM] ...Found 33410 objects in 34 frames.


Measuring Mtb properties of each cell:   0%|          | 0/33410 [00:00<?, ?it/s]

[INFO][2023/10/09 12:24:45 PM] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/10/09 12:24:45 PM] Starting BayesianTracker session
[INFO][2023/10/09 12:24:45 PM] Loading configuration file: /home/dayn/analysis/models/loc.json
[INFO][2023/10/09 12:24:45 PM] Objects are of type: <class 'list'>
[INFO][2023/10/09 12:24:46 PM] Starting tracking... 
[INFO][2023/10/09 12:24:46 PM] Update using: ['VISUAL', 'MOTION']
[INFO][2023/10/09 12:24:46 PM] Tracking objects in frames 0 to 9 (of 34)...
[INFO][2023/10/09 12:24:59 PM]  - Timing (Bayesian updates: 679.81ms, Linking: 10.28ms)
[INFO][2023/10/09 12:24:59 PM]  - Probabilities (Link: 1.00000, Lost: 0.99307)
[INFO][2023/10/09 12:24:59 PM]  - Stats (Active: 1466, Lost: 223, Conflicts resolved: 859)
[INFO][2023/10/09 12:24:59 PM] Tracking objects in frames 10 to 19 (of 34)...
[INFO][2023/10/09 12:25:17 PM]  - Timing (Bayesian updates: 457.67ms, Linking: 9.72ms)
[INFO][2023/10/09 12:25:17 PM]  - Probabilities (Link: 1.

GLPK Integer Optimizer 5.0
11776 rows, 8842 columns, 11796 non-zeros
8842 integer variables, all of which are binary
Preprocessing...
5888 rows, 8842 columns, 11796 non-zeros
8842 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 5888
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
5888 rows, 8842 columns, 11796 non-zeros
*     0: obj =   2.684684813e+04 inf =   0.000e+00 (1176)
*  1180: obj =   1.844015803e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  1180: mip =     not found yet >=              -inf        (1; 0)
+  1180: >>>>>   1.844015803e+04 >=   1.844015803e+04   0.0% (1; 0)
+  1180: mip =   1.844015803e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND


[INFO][2023/10/09 12:25:28 PM] Ending BayesianTracker session
[INFO][2023/10/09 12:25:28 PM] Opening HDF file: /home/dayn/data/homuncu_loc_temp/seperate_tracks/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/sc_analyses/20230802_20X_23-03-001A2_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230802_1504_mphi.h5...
[INFO][2023/10/09 12:25:28 PM] Writing objects/obj_type_1
[INFO][2023/10/09 12:25:28 PM] Writing labels/obj_type_1
[INFO][2023/10/09 12:25:28 PM] Loading objects/obj_type_1 (28388, 5) (28388 filtered: None)
[INFO][2023/10/09 12:25:29 PM] Writing properties/obj_type_1/axis_major_length (28388,)
[INFO][2023/10/09 12:25:29 PM] Writing properties/obj_type_1/axis_minor_length (28388,)
[INFO][2023/10/09 12:25:29 PM] Writing properties/obj_type_1/eccentricity (28388,)
[INFO][2023/10/09 12:25:29 PM] Writing properties/obj_type_1/area (28388,)
[INFO][2023/10/09 12:25:29 PM] Writing properties/obj_type_1/orientation (28388,)
[INFO][2023/10/09 12:25:29 PM] Writing properties/obj_type_1/

In [17]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim=2)

In [18]:
v = napari.Viewer(title = ID_extractor(image_fn))

v.add_image(image, channel_axis=-1)
v.add_labels(masks)
v.add_tracks(napari_tracks)

<Tracks layer 'napari_tracks' at 0x7f86dc27b790>

labels.py (966): Contours are not displayed during 3D rendering
